In [1]:
state = 'NM' 
year = 2020
obj_type = 'average_Polsby_Popper'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of NM has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from NM_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 149 plans from file.


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 698783 and U = 712899 and k = 3.

****************************************
Trying deviation = 7058.406666666667
****************************************
Using user-provided warm starts.
Selected warm_start = [[13, 25], [1, 3, 5, 6, 9, 10, 14, 15, 16, 17, 18, 22, 26, 28, 29, 32], [0, 2, 4, 7, 8, 11, 12, 19, 20, 21, 23, 24, 27, 30, 31]]
Objective value: 0.43342041766639655
Deviation: 5433.333333333372

****************************************
Running labeling model!
****************************************
L = 698783 and U = 712899
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Applying warm start!
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter I

Thread count was 20 (of 20 available processors)

Solution count 3: 0.425766 0.401067 0.3594 
No other solutions better than 0.425766

Optimal solution found (tolerance 0.00e+00)
Best objective 4.257659912109e-01, best bound 4.257659912109e-01, gap 0.0000%

User-callback calls 18765, time in user-callback 0.21 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[13, 18, 25], [1, 3, 5, 6, 9, 10, 14, 15, 16, 17, 22, 26, 28, 29, 32], [0, 2, 4, 7, 8, 11, 12, 19, 20, 21, 23, 24, 27, 30, 31]]

****************************************
Trying deviation = 3208.4999999999613
****************************************
Using user-provided warm starts.
Selected warm_start = [[13, 18, 25], [1, 3, 5, 6, 9, 10, 14, 15, 17, 22, 26, 28, 29, 32], [0, 2, 4, 7, 8, 11, 12, 16, 19, 20, 21, 23, 24, 27, 30, 31]]
Objective value: 0.415501415678718
Deviation: 1510.6666666666279

****************************************
Running labe

     0     2    0.94779    0  230    0.35677    0.94779   166%     -    1s
  2504   735    0.37116   33   90    0.35677    0.61572  72.6%  45.6    5s
  9980  1637    0.41580   36   27    0.35677    0.42642  19.5%  42.7   10s
 20216  1957 infeasible   41         0.35677    0.40036  12.2%  37.7   15s
 28868  1918     cutoff   41         0.35677    0.38017  6.56%  36.2   20s
 43101   626 infeasible   40         0.35677    0.36970  3.62%  35.0   25s

Explored 48482 nodes (1643303 simplex iterations) in 26.71 seconds (7.75 work units)
Thread count was 20 (of 20 available processors)

Solution count 3: 0.356771 0.355463 0.345046 
No other solutions better than 0.356771

Optimal solution found (tolerance 0.00e+00)
Best objective 3.567714691162e-01, best bound 3.567714691162e-01, gap 0.0000%

User-callback calls 98204, time in user-callback 0.57 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[13, 18, 25], 

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
no_solution_region = [0, min(round(r[2],1) for r in result)]
print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)